In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import psycopg2

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np

topics = pd.read_csv('/content/drive/MyDrive/new-mind/topics.csv')
opinions = pd.read_csv('/content/drive/MyDrive/new-mind/opinions.csv')
conclusions = pd.read_csv('/content/drive/MyDrive/new-mind/conclusions.csv')

In [ ]:
target = topics.iloc[1].topic_id

topic = topics[topics['topic_id'] == target].text.values[0]
text = opinions[opinions['topic_id'] == target].text.values
predicted_class = opinions[opinions['topic_id'] == target].type.values

len(text), predicted_class.shape

(8, (8,))

# Option 1: GPT2 Generator pipeline

In [ ]:
generator = pipeline('text-generation', model='gpt2')

In [ ]:
def summarize_text(target):

  topic = topics[topics['topic_id'] == target].text.values[0]
  text = opinions[opinions['topic_id'] == target].text.values
  predicted_class = opinions[opinions['topic_id'] == target].type.values

  prompt = f'Topic: {topic}\n'
  # Generate summary
  for i in range(len(text)):
    prompt += f'{predicted_class[i]}= ' + text[i] + '\n'

  prompt +='Based on that, the conclusion:'

  output = generator(prompt, min_new_tokens=50, max_new_tokens=200)
  output = output[0]['generated_text']

  conc_ind = output.find('conclusion:')
  output = output[conc_ind + 6:]

  return output

In [ ]:
target = topics.iloc[5].topic_id
conclusion = summarize_text(target)
conclusion

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
conclusions[conclusions['topic_id'] == target].text.values[0]

"It is important to keep students' minds engaged over the summer break, so they don't feel completely disoriented the following school year. I know from personal experience that the break between school years has left me disinterested in continuing to learn and work for the next school year. However, giving students teacher-designed assignments will stimulate their brains and keep them focused over the break. "

# Option 2: BART Summarizer pipeline

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
pipe(prompt)

[{'summary_text': 'Cities such as Vauban and Bogota have embraced a no car lifestyle. Car-pooling could significantly reduce emissions as there would be less cars out there. Air pollution can affect those with pre-existing respitory issues. Stress on the road can lead to unsafe driving and a worse performance at work.'}]

In [ ]:
conclusions[conclusions['topic_id'] == target].text.values[0]

"Individual car use isn't bad. Millions of individuals using cars, however, is very bad. Note the emphasis. Even just making a concious effort to car pool, bike to work or school, use public transportation, or use some sort of alternative transportation could be enough to clear up the air of some of it's emissions, keep oneself and others happier and less stressed, save time and money, and improve their city all in one little decision. It's just a matter of going ahead and doing it, so what is stopping anyone from going out and making that change now? Be the change you want to see and limit your car usage now.  "

# Possible Solution 3: Finetuning Pretrained LLM